# AutoGen Other Agents

Samples of other preset agents

In [1]:
%pip install "autogen-ext[openai]==0.4.0.dev13"

Note: you may need to restart the kernel to use updated packages.


## Load Azure Configuration

In [2]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [3]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)


# Multimodal Web Surfer
MultimodalWebSurfer is a multimodal agent that acts as a web surfer that can search the web and visit web pages.

Note: Need to run in terminal: playwright install

In [4]:
# Install the library
%pip install "autogen-ext[web-surfer]==0.4.0.dev13"

  Using cached pyee-12.0.0-py3-none-any.whl.metadata (2.8 kB)
Using cached pyee-12.0.0-py3-none-any.whl (14 kB)
  Attempting uninstall: pyee
    Found existing installation: pyee 13.0.0
    Uninstalling pyee-13.0.0:
      Successfully uninstalled pyee-13.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiortc 1.11.0 requires pyee>=13.0.0, but you have pyee 12.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from autogen_agentchat.ui import Console
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from datetime import datetime

# Get the current date and time
now = datetime.now()
# Extract the date part
current_date = now.date()
print("Current Date:", current_date)

# Define an agent
web_surfer_agent = MultimodalWebSurfer(
    name="MultimodalWebSurfer",
    model_client=az_model_client,
)

# Define a team
agent_team = RoundRobinGroupChat([web_surfer_agent], max_turns=3)

# Run the team and stream messages to the console
stream = agent_team.run_stream(task=f"Today is {current_date}. What is the weather like in New York?")
await Console(stream)
# Close the browser controlled by the agent
await web_surfer_agent.close()


Current Date: 2025-04-17
---------- user ----------
Today is 2025-04-17. What is the weather like in New York?


/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:412: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["vision"] is False:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:427: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["json_output"] is False and json_output is True:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:433: DeprecationWarning: capabilities is deprecated, use model_info instead
  if self.capabilities["function_calling"] is False and len(tools) > 0:
/home/codespace/.python/current/lib/python3.12/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:503: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.

---------- MultimodalWebSurfer ----------
I typed 'current weather in New York' into '0 characters out of 2000'.

 Here is a screenshot of the webpage: [current weather in New York - Search](https://www.bing.com/search?q=current+weather+in+New+York&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=19A991339B1947D6BEF7FF2916D62DB7).
 The viewport shows 29% of the webpage, and is positioned at the top of the page 
The following metadata was extracted from the webpage:

{
    "meta_tags": {
        "referrer": "origin-when-cross-origin",
        "SystemEntropyOriginTrialToken": "A5is4nwJJVnhaJpUr1URgj4vvAXSiHoK0VBbM9fawMskbDUj9WUREpa3JzGAo6xd1Cp2voQEG1h6NQ71AsMznU8AAABxeyJvcmlnaW4iOiJodHRwczovL3d3dy5iaW5nLmNvbTo0NDMiLCJmZWF0dXJlIjoiTXNVc2VyQWdlbnRMYXVuY2hOYXZUeXBlIiwiZXhwaXJ5IjoxNzUzNzQ3MjAwLCJpc1N1YmRvbWFpbiI6dHJ1ZX0=",
        "ConfidenceOriginTrialToken": "Aqw360MHzRcmtEVv55zzdIWcTk2BBYHcdBAOysNJZP4qkN8M+5vUq36ITHFVst8LiX36KBZJXB8xvyBgdK2z5Q0AAAB6eyJvcmlnaW4iOiJodHRwczovL2JpbmcuY29tOjQ0MyI

## OpenAI Assistant Agent

An agent implementation that uses the OpenAI Assistant API to generate responses.

This agent leverages the OpenAI Assistant API to create AI assistants with capabilities like:
- Code interpretation and execution
- File handling and search
- Custom function calling
- Multi-turn conversations

The agent maintains a thread of conversation and can use various tools including
- Code interpreter: For executing code and working with files
- File search: For searching through uploaded documents
- Custom functions: For extending capabilities with user-defined tools

Key Features:
- Supports multiple file formats including code, documents, images
- Can handle up to 128 tools per assistant
- Maintains conversation context in threads
- Supports file uploads for code interpreter and search
- Vector store integration for efficient file search
- Automatic file parsing and embedding

## File Search

In [9]:
from autogen_core import CancellationToken
from autogen_ext.agents.openai import OpenAIAssistantAgent
from autogen_agentchat.messages import TextMessage
from openai import AsyncAzureOpenAI

cancellation_token = CancellationToken()

# Create AzureOpenAI client
client = AsyncAzureOpenAI(azure_endpoint=azure_openai_endpoint, 
                          api_version=azure_openai_api_version, 
                          api_key=azure_openai_key)


#### Create a vector store for file search ####
vector_store = await client.vector_stores.create(name="Hiking Products")
# Specify the folder containing the files
folder_path = "../Data/products/"
# Get all file paths in the folder
file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]
# Open file streams
file_streams = [open(path, "rb") for path in file_paths]
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = await client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)
# Close file streams
for file in file_streams:
    file.close()


/tmp/ipykernel_123705/252920783.py:21: ResourceWarning: unclosed file <_io.BufferedReader name='../Data/products/product_info_2.pdf'>
  file_streams = [open(path, "rb") for path in file_paths]
/tmp/ipykernel_123705/252920783.py:21: ResourceWarning: unclosed file <_io.BufferedReader name='../Data/products/product_info_11.pdf'>
  file_streams = [open(path, "rb") for path in file_paths]
/tmp/ipykernel_123705/252920783.py:21: ResourceWarning: unclosed file <_io.BufferedReader name='../Data/products/product_info_15.pdf'>
  file_streams = [open(path, "rb") for path in file_paths]
/tmp/ipykernel_123705/252920783.py:21: ResourceWarning: unclosed file <_io.BufferedReader name='../Data/products/product_info_12.pdf'>
  file_streams = [open(path, "rb") for path in file_paths]
/tmp/ipykernel_123705/252920783.py:21: ResourceWarning: unclosed file <_io.BufferedReader name='../Data/products/product_info_18.pdf'>
  file_streams = [open(path, "rb") for path in file_paths]
/tmp/ipykernel_123705/252920783

In [10]:
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)
print(vector_store.id)

completed
FileCounts(cancelled=0, completed=20, failed=0, in_progress=0, total=20)
vs_vOgF10qjEWCCAtOybyIdNOs5


In [11]:
#### Create an assistant with file search ####
assistant = OpenAIAssistantAgent(
    name="DataAssistant",
    description="Answer questions about data",
    client=client,
    model=azure_openai_deployment,
    instructions="You will answer questions about the product files given to you",
    tools=["file_search"],
    tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}}
)

# Get response from the assistant
user_input = TextMessage(source="user", content="What is the price of the Adventure Dining Table in the file search product set?")
response = await assistant.on_messages([user_input], cancellation_token
)
print(response.chat_message.content)

# Clean up resources
await assistant.delete_uploaded_files(cancellation_token)
await assistant.delete_assistant(cancellation_token)



The price of the Adventure Dining Table is $90【4:0†source】.


## Code Interpreter

In [12]:
#### Create a file for code interpreter ####
file=open("../Data/assistant/bigfootsightings.csv", "rb")
code_interpreter_file = await client.files.create(
  file=file,
  purpose='assistants'
)
file.close()

#### Create an assistant with code interpreter ####
assistant = OpenAIAssistantAgent(
    name="DataAssistant",
    description="Answer questions about data",
    client=client,
    model=azure_openai_deployment,
    instructions="You will use Python to answer questions about a dataset",
    tools=["code_interpreter"],
    tool_resources={"code_interpreter":{"file_ids":[code_interpreter_file.id]}}
)

# Get response from the assistant
user_input = TextMessage(source="user", content="""Use the code interpreter file. 
                         Which state has the most sightings? Provide the number of sightings in that state""")
response = await assistant.on_messages([user_input], cancellation_token
)
print(response.chat_message.content)

# Clean up resources
await assistant.delete_uploaded_files(cancellation_token)
await assistant.delete_assistant(cancellation_token)

The state with the most sightings is Washington, with a total of 601 sightings.


## Code Executor Agent
An agent that extracts and executes code snippets found in received messages and returns the output.

It is typically used within a team with another agent that generates code snippets to be executed.

In [13]:
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor
from autogen_core import CancellationToken


# Create a code executor agent that uses a Docker container to execute code.
docker_code_executor = DockerCommandLineCodeExecutor(work_dir="coding")

# Start the docker code executor.
await docker_code_executor.start()

# Create a code executor agent.
code_executor_agent = CodeExecutorAgent("code_executor", code_executor=docker_code_executor)

# Run the agent with a given code snippet.
task = TextMessage(
    content='''Here is some code
    ```python
print('Hello world')
    ```
    ''',
    source="user",
)

response = await code_executor_agent.on_messages([task], CancellationToken())
print(response.chat_message)

# Stop the docker code executor.
await docker_code_executor.stop()


source='code_executor' models_usage=None content='Hello world\n' type='TextMessage'
